# Get Topics (Skills), Words (sub-skills)
- Richard Kuzma, 23SEP2020

### imports

In [1]:
# basic
from pprint import pprint
import pickle
import time

# data science
import pandas as pd
import numpy as np
print(np.__version__)

# NLP
import gensim
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from gensim.test.utils import datapath

# plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline


### Import preprocesss_helpers module
import os
import sys

# add '/Users/richardkuzma/coding/analysis' to path
# where /utils holds module preprocess_helpers    
module_path = os.path.abspath(os.path.join('../..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.preprocess_helpers as ph

1.15.4


/Users/richardkuzma/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/richardkuzma/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/richardkuzma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/richardkuzma/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### methods

In [2]:
def visualize_lda(model, corpus, dictionary):
    """returns the pyLDAvis PreparedData given model, corpus, dictionary"""
    """Could pickle this to save it"""
    """pyLDAvis.save_html(vis, "filename") also works to export in html"""
    pyLDAvis.enable_notebook()
    t0 = time.time()
    vis = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    print('{} seconds'.format(time.time()-t0))
    return vis

def gather_words_from_LDA_topics(model):
    """top 100 words associated with each topic"""
    topic_words = []
    total_missed_words = 0
    words_per_topic = 100
    for i in range (0, model.num_topics): # for each LDA topic
        missed_words = 0
        this_topic_words = []
        for j in range(0, words_per_topic): # for first 'words_per_topic' words in a topic
            try:
                this_topic_words.append(model.show_topic(i, topn=words_per_topic)[j][0])
            except KeyError:
                missed_words +=1 

        total_missed_words += missed_words
        topic_words.append(this_topic_words)
    return topic_words



def display_topic_and_words(topic_num, num_words, topic_names):
    """display top num_words words from each topic"""
    print('Topic name: {}'.format(topic_names[topic_num]))
    print('Topic words: {}'.format(topic_words[topic_num][:num_words]))
    print()


    
### generate 'skill' topics for document, with weights
### note, not all topics may be represented
def get_doc_topics(document_string: str, model):

    # turn string into list of length 1
    doc_string = [document_string]
    
    # clean text into lemmatized lists of unigrams, bigrams, trigrams
    doc_cleaned = ph.full_clean(doc_string)

    # create dictionary
    doc_dict = ph.make_dict(doc_cleaned)

    # make corpus
    doc_bow = ph.make_bow(doc_cleaned, doc_dict)

    # create lda probability distribution of topics
    # doc_lda is a gensim transformed corpus
    doc_lda = model[doc_bow]
    
    # only one document means only one element, a list of tuples with (topic, probability)    
    for topic in doc_lda:
        doc_topics = topic
        # there should only be one list, so break
        break 
    
    # list of tuples
    return doc_topics
    



# order top 'skills' of a document
def get_doc_top_topics(doc_topic_list: list):

    doc_topic_list.sort(reverse=True, key=lambda x: x[1])
    
    ordered_topic_list = []
    for i in range(0, len(doc_topic_list)):
        ordered_topic_list.append(doc_topic_list[i][0])
    
    return ordered_topic_list
    
#     e.g. input:  [(6, 0.09539536), (8, 0.045605347), (24, 0.23120484), (32, 0.19147022), (34, 0.40980446)]
#          output: [34, 24, 32, 6, 8]


### Load corpus and dict 
(already created)

In [3]:
path = '/Users/richardkuzma/coding/analysis/monster/data/cleaned/'

# jobs_cleaned_filename = 'monster_jobs_cleaned_text.pkl'
# with open(path+jobs_cleaned_filename, 'rb') as f:
#     jobs_cleaned = pickle.load(f)

jobs_dict_filename = 'monster_jobs_dict.pkl'
with open(path+jobs_dict_filename, 'rb') as f:
    jobs_dict = pickle.load(f)
    
jobs_corpus_filename = 'monster_jobs_corpus.pkl'
with open(path+jobs_corpus_filename, 'rb') as f:
    jobs_corpus = pickle.load(f)



## Create LDA model

In [4]:
model = LdaModel(corpus=jobs_corpus, num_topics=40, id2word=jobs_dict)

print('Numpy version: {}'.format(np.version.version))
# important, made with numpy 1.15.4 even though my env numpy is 1.19.2
# if you have different numpy version, you can't load this?

Numpy version: 1.15.4


In [5]:
# Save model to disk.
model_path = '/Users/richardkuzma/coding/analysis/monster/models/'
model_name = 'LDA_40topics_23SEP2020.model'
model.save(model_path+model_name)


In [6]:
# Load from disk.
model = LdaModel.load(model_path+model_name)

In [7]:
pprint(model.print_topics())

[(36,
  '0.041*"sale" + 0.036*"marketing" + 0.020*"customer" + 0.017*"level" + '
  '0.015*"client" + 0.015*"management" + 0.014*"business" + 0.013*"entry" + '
  '0.011*"retail" + 0.011*"training"'),
 (27,
  '0.019*"employee" + 0.018*"employment" + 0.012*"information" + '
  '0.009*"ability" + 0.009*"client" + 0.009*"status" + 0.009*"age" + '
  '0.008*"disability" + 0.008*"equal" + 0.007*"national_origin"'),
 (5,
  '0.031*"engineering" + 0.024*"design" + 0.023*"system" + 0.019*"technical" + '
  '0.016*"engineer" + 0.014*"product" + 0.011*"customer" + 0.010*"technology" '
  '+ 0.010*"development" + 0.009*"test"'),
 (6,
  '0.033*"perform" + 0.025*"ability" + 0.019*"duty" + 0.018*"customer" + '
  '0.017*"must" + 0.016*"essential" + 0.013*"equipment" + 0.012*"function" + '
  '0.010*"may" + 0.010*"employee"'),
 (26,
  '0.041*"food" + 0.015*"management" + 0.014*"maintain" + 0.012*"ensure" + '
  '0.012*"customer" + 0.011*"ability" + 0.010*"equipment" + 0.009*"area" + '
  '0.009*"business" + 0.0

In [8]:
visualize_lda(model, corpus=jobs_corpus, dictionary=jobs_dict)

47.20770311355591 seconds


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2      0.084969 -0.056356       1        1  6.352745
25     0.052307 -0.137024       2        1  5.962257
35     0.115607 -0.114878       3        1  5.623957
14    -0.083039 -0.051738       4        1  5.230800
31    -0.010075 -0.107239       5        1  4.748574
36     0.023358  0.107909       6        1  4.389521
11     0.017938  0.088281       7        1  3.993110
28    -0.034236 -0.009147       8        1  3.889579
18     0.039660  0.029354       9        1  3.626708
22     0.105393 -0.106034      10        1  3.612906
9      0.100584 -0.096370      11        1  3.524327
37     0.023783 -0.075437      12        1  3.438409
24     0.044362  0.173345      13        1  3.179209
5      0.111132 -0.074520      14        1  3.164738
17     0.050809 -0.053823      15        1  3.077130
27     0.065374 -0.071769      16        1  3.003157
6     -0.044039 -0.077125      17        1  2.882137
39     0.030740 -0.030838      18        1  2.245246
21    -0.408425 -0.101135      19        1  2.209994
19     0.037580 -0.039270      20        1  2.024346
4      0.029732  0.119458      21        1  1.974037
26    -0.027760 -0.111177      22        1  1.929220
23    -0.337686 -0.154883      23        1  1.867632
16     0.134041  0.006188      24        1  1.861342
38     0.040891  0.182749      25        1  1.841546
0      0.005088  0.043451      26        1  1.625640
15    -0.065468 -0.015864      27        1  1.492163
32     0.091965 -0.009153      28        1  1.438554
13     0.045326 -0.059519      29        1  1.158320
12    -0.053145  0.054316      30        1  1.142437
10    -0.087054  0.122418      31        1  1.097936
20     0.011794  0.024855      32        1  1.090004
7     -0.066389  0.080832      33        1  0.945840
34    -0.099593  0.282948      34        1  0.898086
1     -0.097874  0.043015      35        1  0.802908
3      0.036534  0.173769      36        1  0.723349
33     0.084561 -0.014262      37        1  0.562513
8      0.031384 -0.037863      38        1  0.507063
29    -0.084499  0.126428      39        1  0.497833
30     0.084370 -0.053889      40        1  0.364722, topic_info=               Term          Freq         Total Category  logprob  loglift
653           store  19586.000000  19586.000000  Default  30.0000  30.0000
625         manager  27969.000000  27969.000000  Default  29.0000  29.0000
210            care  13594.000000  13594.000000  Default  28.0000  28.0000
530            sale  21185.000000  21185.000000  Default  27.0000  27.0000
91          project  22536.000000  22536.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
809         storage     63.657444   1160.201416  Topic40  -5.3921   2.7110
1040       security     72.029190   6819.129395  Topic40  -5.2685   1.0634
286       knowledge     73.908913  16275.735352  Topic40  -5.2427   0.2192
212   certification     64.867104   4402.486816  Topic40  -5.3732   1.3962
72             must     64.425369  23208.980469  Topic40  -5.3801  -0.2730

[3118 rows x 6 columns], token_table=      Topic      Freq                              Term
term                                                   
6142     17  0.988014          abilities_close_distance
8221      2  0.168237      abilities_typically_acquired
8221     29  0.824902      abilities_typically_acquired
185       1  0.061983                           ability
185       2  0.039347                           ability
...     ...       ...                               ...
1668     31  0.982652  youll_designing_building_bridges
7705     28  0.992551                             youth
4209     11  0.008685                 youtube_test_href
4209     12  0.004342                 youtube_test_href
4209     38  0.972673                 youtube_test_href

[23685 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xl

## Load jobs df

In [9]:
df_path = '/Users/richardkuzma/coding/analysis/monster/data/cleaned/'
df_filename = 'monster_jobs_df_small.pkl'
with open(df_path+df_filename, 'rb') as f:
    df = pickle.load(f)
df.head()

,id,job_title,job_description,job_type,location,organization,sector
0,0,IT Support Technician Job in Madison,TeamSoft is seeing an IT Support Specialist to...,Full Time Employee,"Madison, WI 53702",None,IT/Software Development
1,1,Business Reporter/Editor Job in Madison,The Wisconsin State Journal is seeking a flexi...,Full Time,"Madison, WI 53708",Printing and Publishing,None
2,2,Johnson & Johnson Family of Companies Job Appl...,Report this job About the Job DePuy Synthes Co...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,None
3,3,Engineer - Quality Job in Dixon,Why Join Altec? If you’re considering a career...,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager)
4,4,Shift Supervisor - Part-Time Job in Camphill,Position ID# 76162 # Positions 1 State CT C...,Full Time Employee,"Camphill, PA",Retail,Project/Program Management


## I make up topic names

In [10]:
### my unscientific labeling of topics
topic_names = ["Office Administrator", "Customer Support", "Business Analyst", "Business Process Manager", "Hiring Manager",
               "Customer Service", "Financial Accountant", "Information Technology", "Food Service", "Misc 1",
               "Sales", "Misc 2", "Retail Support", "Project Manager 1",  "Construction", 
               "Equal Opportunity", "Testing and Evaluation", "Process Manager", "Medical", "Project Manager 2",
               "Quality Assurance", "Training", "IT/Software Engineering", "Sales/Marketing", "Product / Business",
               "Restaurant Management", "Store Manager", "Customer Service", "Driver", "Maintenance / Repair",
               "Driving / Transportation", "Real Estate", "Financial Education", "Physical Security", "Misc 3", 
               "Insurance", "Clinical / Medical", "Staffing / Training / Mgmt", "Nursing / Medical", "Customer Success / Product"]


In [11]:
topic_words = gather_words_from_LDA_topics(model)

for i in range(0, model.num_topics):
    display_topic_and_words(i, 10, topic_names=topic_names)

Topic name: Office Administrator
Topic words: ['restaurant', 'manager', 'management', 'company', 'must', 'great', 'kitchen', 'general', 'hour', 'high']

Topic name: Customer Support
Topic words: ['care', 'home', 'patient', 'nurse', 'clinical', 'manager', 'rn', 'case', 'medical', 'registered_nurse']

Topic name: Business Analyst
Topic words: ['management', 'business', 'development', 'client', 'program', 'leadership', 'develop', 'manage', 'manager', 'ability']

Topic name: Business Process Manager
Topic words: ['real_estate', 'new', 'company', 'loan', 'career', 'family', 'lease', 'today', 'time', 'currently']

Topic name: Hiring Manager
Topic words: ['must', 'time', 'hour', 'full', 'candidate', 'able', 'benefit', 'available', 'apply', 'please']

Topic name: Customer Service
Topic words: ['engineering', 'design', 'system', 'technical', 'engineer', 'product', 'customer', 'technology', 'development', 'test']

Topic name: Financial Accountant
Topic words: ['perform', 'ability', 'duty', 'cust

### Example:

In [12]:
ex = "I'm a nurse. Nursing medical doctor physician doctor doctor doctor needle. Hospital patient care. nurse nurse nurse nurse nurse Technology programming software engineer is my habit. C++. Customer service is my number one priority. business accounting marketing finance"


ex_topics = get_doc_topics(ex, model)
print('\nExample doc_topics:')
print(ex_topics)


ordered_ex_topics = get_doc_top_topics(ex_topics)
print('\nExample doc_top_topics')
print(ordered_ex_topics)

lemmatizing...
lemmatization complete
2.69500470161438 seconds

Example doc_topics:
[(18, 0.13999254), (20, 0.29912877), (24, 0.10822819), (35, 0.10930417), (38, 0.31682563)]

Example doc_top_topics
[38, 20, 18, 35, 24]


In [13]:
# identify topics for each job post document 
# this took >35min
df['doc_topics'] = df.apply(lambda x: get_doc_topics(x['job_description'], model), axis=1)

# order topics
df['doc_top_topics'] = temp.apply(lambda x: get_doc_top_topics(x['doc_topics']), axis=1)

lemmatizing...
lemmatization complete
0.051088809967041016 seconds
lemmatizing...
lemmatization complete
0.021699905395507812 seconds
lemmatizing...
lemmatization complete
0.15149402618408203 seconds
lemmatizing...
lemmatization complete
0.11505293846130371 seconds
lemmatizing...
lemmatization complete
0.1608879566192627 seconds
lemmatizing...
lemmatization complete
0.03030991554260254 seconds
lemmatizing...
lemmatization complete
0.07146883010864258 seconds
lemmatizing...
lemmatization complete
0.05582284927368164 seconds
lemmatizing...
lemmatization complete
0.07848286628723145 seconds
lemmatizing...
lemmatization complete
0.09253382682800293 seconds
lemmatizing...
lemmatization complete
0.10631036758422852 seconds
lemmatizing...
lemmatization complete
0.022867918014526367 seconds
lemmatizing...
lemmatization complete
0.03386497497558594 seconds
lemmatizing...
lemmatization complete
0.043534040451049805 seconds
lemmatizing...
lemmatization complete
0.027985811233520508 seconds
lemmat

lemmatization complete
0.18469595909118652 seconds
lemmatizing...
lemmatization complete
0.008925914764404297 seconds
lemmatizing...
lemmatization complete
0.026297807693481445 seconds
lemmatizing...
lemmatization complete
0.07417798042297363 seconds
lemmatizing...
lemmatization complete
0.024236202239990234 seconds
lemmatizing...
lemmatization complete
0.15650105476379395 seconds
lemmatizing...
lemmatization complete
0.06788992881774902 seconds
lemmatizing...
lemmatization complete
0.03780388832092285 seconds
lemmatizing...
lemmatization complete
0.0026459693908691406 seconds
lemmatizing...
lemmatization complete
0.03487586975097656 seconds
lemmatizing...
lemmatization complete
0.04327583312988281 seconds
lemmatizing...
lemmatization complete
0.03250408172607422 seconds
lemmatizing...
lemmatization complete
0.04456901550292969 seconds
lemmatizing...
lemmatization complete
0.05133199691772461 seconds
lemmatizing...
lemmatization complete
0.021136999130249023 seconds
lemmatizing...
lemm

lemmatizing...
lemmatization complete
0.1098330020904541 seconds
lemmatizing...
lemmatization complete
0.048486948013305664 seconds
lemmatizing...
lemmatization complete
0.10669326782226562 seconds
lemmatizing...
lemmatization complete
0.0547487735748291 seconds
lemmatizing...
lemmatization complete
0.031133174896240234 seconds
lemmatizing...
lemmatization complete
0.03057098388671875 seconds
lemmatizing...
lemmatization complete
0.09730195999145508 seconds
lemmatizing...
lemmatization complete
0.02882099151611328 seconds
lemmatizing...
lemmatization complete
0.027070283889770508 seconds
lemmatizing...
lemmatization complete
0.10531282424926758 seconds
lemmatizing...
lemmatization complete
0.06847071647644043 seconds
lemmatizing...
lemmatization complete
0.047499895095825195 seconds
lemmatizing...
lemmatization complete
0.0474858283996582 seconds
lemmatizing...
lemmatization complete
0.0038797855377197266 seconds
lemmatizing...
lemmatization complete
0.03339505195617676 seconds
lemmati

lemmatizing...
lemmatization complete
0.06664490699768066 seconds
lemmatizing...
lemmatization complete
0.045606136322021484 seconds
lemmatizing...
lemmatization complete
0.03508877754211426 seconds
lemmatizing...
lemmatization complete
0.04692721366882324 seconds
lemmatizing...
lemmatization complete
0.04121708869934082 seconds
lemmatizing...
lemmatization complete
0.0074939727783203125 seconds
lemmatizing...
lemmatization complete
0.04191923141479492 seconds
lemmatizing...
lemmatization complete
0.020340919494628906 seconds
lemmatizing...
lemmatization complete
0.036392927169799805 seconds
lemmatizing...
lemmatization complete
0.08042192459106445 seconds
lemmatizing...
lemmatization complete
0.0511171817779541 seconds
lemmatizing...
lemmatization complete
0.04433417320251465 seconds
lemmatizing...
lemmatization complete
0.010717153549194336 seconds
lemmatizing...
lemmatization complete
0.011523008346557617 seconds
lemmatizing...
lemmatization complete
0.0015189647674560547 seconds
le

lemmatizing...
lemmatization complete
0.056215763092041016 seconds
lemmatizing...
lemmatization complete
0.04671001434326172 seconds
lemmatizing...
lemmatization complete
0.036142826080322266 seconds
lemmatizing...
lemmatization complete
0.0681612491607666 seconds
lemmatizing...
lemmatization complete
0.036949872970581055 seconds
lemmatizing...
lemmatization complete
0.03535008430480957 seconds
lemmatizing...
lemmatization complete
0.049549102783203125 seconds
lemmatizing...
lemmatization complete
0.03888511657714844 seconds
lemmatizing...
lemmatization complete
0.05810689926147461 seconds
lemmatizing...
lemmatization complete
0.03819584846496582 seconds
lemmatizing...
lemmatization complete
0.1676468849182129 seconds
lemmatizing...
lemmatization complete
0.029890060424804688 seconds
lemmatizing...
lemmatization complete
0.03215384483337402 seconds
lemmatizing...
lemmatization complete
0.030071020126342773 seconds
lemmatizing...
lemmatization complete
0.017260074615478516 seconds
lemma

...

lemmatization complete
0.012312173843383789 seconds
lemmatizing...
lemmatization complete
0.009364128112792969 seconds
lemmatizing...
lemmatization complete
0.03312802314758301 seconds
lemmatizing...
lemmatization complete
0.0414888858795166 seconds
lemmatizing...
lemmatization complete
0.031339168548583984 seconds
lemmatizing...
lemmatization complete
0.055264949798583984 seconds
lemmatizing...
lemmatization complete
0.026188135147094727 seconds
lemmatizing...
lemmatization complete
0.08974123001098633 seconds
lemmatizing...
lemmatization complete
0.04029202461242676 seconds
lemmatizing...
lemmatization complete
0.07059907913208008 seconds
lemmatizing...
lemmatization complete
0.028591156005859375 seconds
lemmatizing...
lemmatization complete
0.03169703483581543 seconds
lemmatizing...
lemmatization complete
0.014849185943603516 seconds
lemmatizing...
lemmatization complete
0.05016279220581055 seconds
lemmatizing...
lemmatization complete
0.017736196517944336 seconds
lemmatizing...
lem

lemmatization complete
0.04681801795959473 seconds
lemmatizing...
lemmatization complete
0.028628826141357422 seconds
lemmatizing...
lemmatization complete
0.02655196189880371 seconds
lemmatizing...
lemmatization complete
0.030000925064086914 seconds
lemmatizing...
lemmatization complete
0.019324302673339844 seconds
lemmatizing...
lemmatization complete
0.04701495170593262 seconds
lemmatizing...
lemmatization complete
0.02756524085998535 seconds
lemmatizing...
lemmatization complete
0.02762603759765625 seconds
lemmatizing...
lemmatization complete
0.04586505889892578 seconds
lemmatizing...
lemmatization complete
0.025845050811767578 seconds
lemmatizing...
lemmatization complete
0.06101083755493164 seconds
lemmatizing...
lemmatization complete
0.028986215591430664 seconds
lemmatizing...
lemmatization complete
0.013576984405517578 seconds
lemmatizing...
lemmatization complete
0.023355960845947266 seconds
lemmatizing...
lemmatization complete
0.028734922409057617 seconds
lemmatizing...
le

lemmatization complete
0.06478095054626465 seconds
lemmatizing...
lemmatization complete
0.041611671447753906 seconds
lemmatizing...
lemmatization complete
0.009305953979492188 seconds
lemmatizing...
lemmatization complete
0.029356002807617188 seconds
lemmatizing...
lemmatization complete
0.053681135177612305 seconds
lemmatizing...
lemmatization complete
0.04088187217712402 seconds
lemmatizing...
lemmatization complete
0.03675389289855957 seconds
lemmatizing...
lemmatization complete
0.02176690101623535 seconds
lemmatizing...
lemmatization complete
0.04368090629577637 seconds
lemmatizing...
lemmatization complete
0.01798105239868164 seconds
lemmatizing...
lemmatization complete
0.04389691352844238 seconds
lemmatizing...
lemmatization complete
0.05027294158935547 seconds
lemmatizing...
lemmatization complete
0.175462007522583 seconds
lemmatizing...
lemmatization complete
0.04685211181640625 seconds
lemmatizing...
lemmatization complete
0.08068299293518066 seconds
lemmatizing...
lemmatiz

lemmatizing...
lemmatization complete
0.04296612739562988 seconds
lemmatizing...
lemmatization complete
0.025025129318237305 seconds
lemmatizing...
lemmatization complete
0.06415581703186035 seconds
lemmatizing...
lemmatization complete
0.02892279624938965 seconds
lemmatizing...
lemmatization complete
0.0745689868927002 seconds
lemmatizing...
lemmatization complete
0.03566122055053711 seconds
lemmatizing...
lemmatization complete
0.06833314895629883 seconds
lemmatizing...
lemmatization complete
0.030707359313964844 seconds
lemmatizing...
lemmatization complete
0.04171919822692871 seconds
lemmatizing...
lemmatization complete
0.02882099151611328 seconds
lemmatizing...
lemmatization complete
0.020474910736083984 seconds
lemmatizing...
lemmatization complete
0.09307003021240234 seconds
lemmatizing...
lemmatization complete
0.016903162002563477 seconds
lemmatizing...
lemmatization complete
0.004848957061767578 seconds
lemmatizing...
lemmatization complete
0.047857046127319336 seconds
lemma

NameError: name 'temp' is not defined

In [14]:
df.head()

,id,job_title,job_description,job_type,location,organization,sector,doc_topics
0,0,IT Support Technician Job in Madison,TeamSoft is seeing an IT Support Specialist to...,Full Time Employee,"Madison, WI 53702",None,IT/Software Development,"[(2, 0.019509971), (4, 0.012648029), (6, 0.025..."
1,1,Business Reporter/Editor Job in Madison,The Wisconsin State Journal is seeking a flexi...,Full Time,"Madison, WI 53708",Printing and Publishing,None,"[(2, 0.2038833), (9, 0.037298), (11, 0.3219967..."
2,2,Johnson & Johnson Family of Companies Job Appl...,Report this job About the Job DePuy Synthes Co...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,None,"[(1, 0.010344258), (2, 0.15930873), (4, 0.0303..."
3,3,Engineer - Quality Job in Dixon,Why Join Altec? If you’re considering a career...,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager),"[(2, 0.1604361), (5, 0.16964124), (6, 0.046052..."
4,4,Shift Supervisor - Part-Time Job in Camphill,Position ID# 76162 # Positions 1 State CT C...,Full Time Employee,"Camphill, PA",Retail,Project/Program Management,"[(2, 0.05624825), (5, 0.010612251), (6, 0.0150..."


In [15]:
### save df
df_path = '/Users/richardkuzma/coding/analysis/monster/data/cleaned/'
df_filename = 'monster_jobs_df_with_topics.pkl'
with open(df_path+df_filename, 'wb') as f:
    pickle.dump(df, f)

In [16]:
### load df
df_path = '/Users/richardkuzma/coding/analysis/monster/data/cleaned/'
df_filename = 'monster_jobs_df_with_topics.pkl'
with open(df_path+df_filename, 'rb') as f:
    df = pickle.load(f)

df.head()

,id,job_title,job_description,job_type,location,organization,sector,doc_topics,doc_top_topics
0,0,IT Support Technician Job in Madison,TeamSoft is seeing an IT Support Specialist to...,Full Time Employee,"Madison, WI 53702",None,IT/Software Development,"[(34, 0.33607256), (24, 0.2506341), (7, 0.1246...","[34, 24, 7, 27, 10, 9, 32, 2]"
1,1,Business Reporter/Editor Job in Madison,The Wisconsin State Journal is seeking a flexi...,Full Time,"Madison, WI 53708",Printing and Publishing,None,"[(34, 0.35871798), (24, 0.22570807), (27, 0.20...","[34, 24, 27, 39, 32]"
2,2,Johnson & Johnson Family of Companies Job Appl...,Report this job About the Job DePuy Synthes Co...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,None,"[(24, 0.17476252), (34, 0.15736698), (37, 0.12...","[24, 34, 37, 19, 27, 7, 29, 28, 16, 2, 31, 9, ..."
3,3,Engineer - Quality Job in Dixon,Why Join Altec? If you’re considering a career...,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager),"[(24, 0.20749241), (28, 0.110996574), (19, 0.1...","[24, 28, 19, 37, 34, 16, 7, 29, 27, 0, 5, 2, 3..."
4,4,Shift Supervisor - Part-Time Job in Camphill,Position ID# 76162 # Positions 1 State CT C...,Full Time Employee,"Camphill, PA",Retail,Project/Program Management,"[(34, 0.35330316), (2, 0.3301695), (24, 0.0752...","[34, 2, 24, 27, 7, 30, 31, 36, 32, 9]"


In [46]:
def show_job_skills(job_num, df=df, words_per_topic=5,  num_topics=40):
    count = 0
    for i in df.iloc[job_num]['doc_top_topics']:
        display_topic_and_words(i, words_per_topic, topic_names)
        count += 1
        if count == num_topics:
            break

In [65]:
def display_job_info(job_num, df=df):
    print('\n')
    print('='*30 + ' Information on Job #{} '.format(job_num) + '='*30)
    print('Job ID: {}'.format(df.iloc[job_num]['id']))
    print('Job Title: {}'.format(df.iloc[job_num]['job_title']))
    print('Organization: {}'.format(df.iloc[job_num]['organization']))
    print('Sector: {}'.format(df.iloc[job_num]['sector']))
    print('Location: {}'.format(df.iloc[job_num]['location']))
    print('Job Type: {}'.format(df.iloc[job_num]['job_type']))
    
    print('\n' + '-'*20 + ' Skills/Topics in order of importance ' + '-'*20 + '\n')
    show_job_skills(job_num, df=df, num_topics = 5, words_per_topic = 10)
    
    print('\n' + '-'*20 + ' Job Description ' + '-'*20 + '\n')
    print(df.iloc[job_num]['job_description'])

## Show skills for any job description

In [66]:
examples = [134, 2345, 5432]
for i in examples:
    display_job_info(i)



============================== Information on Job #134 ==============================
Job ID: 134
Job Title: SOMMELIERS
Organization: All
Sector: Manager (Manager/Supervisor of Staff)
Location: Denver, CO
Job Type: Full Time

-------------------- Skills/Topics in order of importance --------------------

Topic name: Misc 3
Topic words: ['aflac', 'help', 'business', 'company', 'benefit', 'sale', 'insurance', 'one', 'life', 'grow']

Topic name: Product / Business
Topic words: ['company', 'benefit', 'insurance', 'life', 'offer', 'career', 'paid', 'employee', 'package', 'plan']

Topic name: Customer Service
Topic words: ['employee', 'employment', 'information', 'ability', 'client', 'status', 'age', 'disability', 'equal', 'national_origin']

Topic name: Information Technology
Topic words: ['company', 'ability', 'time', 'beacon_hill', 'driver', 'apply', 'pay', 'associate', 'customer', 'perform']

Topic name: Project Manager 2
Topic words: ['accounting', 'financial', 'payroll', 'finance', '